In [1]:
import cdms2 as cdms
import MV2 as MV
import cdtime,cdutil,genutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import string
import glob
import scipy.stats as stats
# Local solution
# If running remotely, uncomment the following code:
# %%bash
# git clone https://github.com/katemarvel/CMIP5_tools
# import CMIP5_tools as cmip5
import sys,os
sys.path.append("/Users/kmarvel/Google Drive/python-utils")

import CMIP5_tools as cmip5
import DA_tools
import Plotting

from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
%matplotlib inline


In [2]:
central_plains=cdutil.region.domain(longitude=(-105,-92),latitude=(32,46))
southwest=cdutil.region.domain(longitude=(-125,-105),latitude=(32,41))


In [3]:
#sshfs mout to kdm2144@dester.ldeo.columbia.edu:/home/kdm2144/ 
datadirec="/Users/kmarvel/Documents/DATA/dester/piControlRaw/"
#datadirec="/Volumes/RAVEN/CMIP6/cmip6rawyr/"
writedirec="/Users/kmarvel/Documents/DATA/dester/regional_averages/"
fixedvardirec="/Users/kmarvel/Documents/DATA/dester/fixedvar/"

In [4]:
variables=[x.split("/")[-1] for x in glob.glob(datadirec+"*")]
variable="pr"
modeldirs=glob.glob(datadirec+variable+"/*")
direc=modeldirs[0]
direc
model=direc.split("/")[-1]
allfiles=glob.glob(direc+"/*"+variable+"*")
allfiles
experiments=np.unique([x.split(".")[1] for x in allfiles])
experiment=experiments[0]
allfiles_experiment=glob.glob(direc+"/"+variable+"."+experiment+".*")
allfiles_experiment
rips=np.unique([x.split(".")[-3] for x in allfiles_experiment])
rip=rips[0]
rip
landthresh=1
landfiles=glob.glob(fixedvardirec+"sftlf*"+model+".*")
fland=cdms.open(landfiles[0])
landfrac=fland("sftlf")
fland.close()
ripfiles=np.sort(glob.glob(direc+"/"+variable+"."+experiment+"."+model+"."+rip+"*"))
ripfile=ripfiles[0]
frip=cdms.open(ripfile)
data=frip(variable)
frip.close()


In [4]:
variables=[x.split("/")[-1] for x in glob.glob(datadirec+"*")]


###### LOOP OVER ALL VARIABLES #####
for variable in variables:
    #print(variable)
    cmd = "mkdir "+writedirec+"/central_plains/"+variable
    os.system(cmd)

    cmd = "mkdir "+writedirec+"/southwest/"+variable
    os.system(cmd)
    
    modeldirs=glob.glob(datadirec+variable+"/*")
    
    ### LOOP OVER ALL MODELS
    for direc in modeldirs:
        model=direc.split("/")[-1]
        #print(model)
        allfiles=glob.glob(direc+"/*"+variable+"*")
        experiments=np.unique([x.split(".")[1] for x in allfiles])
        ###### LOOP OVER ALL EXPERIMENTS #####
        for experiment in experiments:
            #print(experiment)
            cp_writedirec=writedirec+"central_plains/"+variable+"/"+experiment+"/"
            cmd="mkdir "+cp_writedirec
            os.system(cmd)
            sw_writedirec=writedirec+"southwest/"+variable+"/"+experiment+"/"
            cmd="mkdir "+sw_writedirec
            os.system(cmd)
            allfiles_experiment=glob.glob(direc+"/"+variable+"."+experiment+".*")
            rips=np.unique([x.split(".")[-3] for x in allfiles_experiment])

            landthresh=1
            #Get the land fraction
            landfiles=glob.glob(fixedvardirec+"sftlf*"+model+".*")
            

            if len(landfiles)==1:
                fland=cdms.open(landfiles[0])
                landfrac=fland("sftlf")
                fland.close()
            else:
                #print("can't find land fraction file for", model)
                #print(landfiles)
                continue
            #print(direc)



            ###### LOOP OVER ALL RIPS #####
            for rip in rips:
                #print(rip)
                cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                if (cp_writename in glob.glob(cp_writedirec+"*")):
                    #print("Already done")
                    continue
                    
                yearcheck=[]
                #ripfiles=sorted(glob.glob(direc+"/*"+rip+"*"))
                ripfiles=np.sort(glob.glob(direc+"/"+variable+"."+experiment+"."+model+"."+rip+"*"))
                print(variable+"."+experiment+"."+model+"."+rip+"*")
                L=len(ripfiles)
                i=0
                ripfile=ripfiles[i]
                frip=cdms.open(ripfile)
                data=frip(variable)
                frip.close()
                if data.shape[1:]!=landfrac.shape:
                    print ("land mask wrong shape for "+variable+"."+experiment+"."+model+"."+rip)
                    continue
                                

                
                latax=landfrac.getLatitude()
                lonax=landfrac.getLongitude()
                #data.setAxis(1,latax)
                #data.setAxis(2,lonax)
                tax=np.arange(12)
                fpath,fexpt,fmodel,frip,fyear,fnc=ripfile.split(".")
                CP=MV.zeros(L*12)
                SW=MV.zeros(L*12)
                #full[12*i:12*(i+1)]=data
                landdata=MV.masked_where(np.repeat(landfrac.asma()[np.newaxis],12,axis=0)<landthresh,data)
                landdata.setAxis(1,latax)
                landdata.setAxis(2,lonax)
                CP[12*i:12*(i+1)]=cdutil.averager(landdata(central_plains),axis='xy')
                SW[12*i:12*(i+1)]=cdutil.averager(landdata(southwest),axis='xy')
                    
                
                yearcheck+=[float(fyear)]
                for i in range(L)[1:]:

                    ripfile=ripfiles[i]
                    
                    f=cdms.open(ripfile)
                    data=f(variable)
                        
                    landdata=MV.masked_where(np.repeat(landfrac.asma()[np.newaxis],12,axis=0)<landthresh,data)
                    f.close()
                    #Kludge since downloading process didn't preserve lat/lon designation
                    landdata.setAxis(1,latax)
                    landdata.setAxis(2,lonax)


                    fpath,fexpt,fmodel,frip,fyear,fnc=ripfile.split(".")
                    CP[12*i:12*(i+1)]=cdutil.averager(landdata(central_plains),axis='xy')
                    SW[12*i:12*(i+1)]=cdutil.averager(landdata(southwest),axis='xy')
                    yearcheck+=[float(fyear)]
                tax=cdms.createAxis(np.arange(L*12))
                tax.designateTime()
                tax.units='months since '+str(yearcheck[0])+'-1-1'
                CP.setAxis(0,tax)
                SW.setAxis(0,tax)
                CP.id=variable
                SW.id=variable
                cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fcpw=cdms.open(cp_writename,"w")    
                fcpw.write(CP)
                fcpw.close()
                sw_writename=sw_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fsww=cdms.open(sw_writename,"w")    
                fsww.write(SW)
                fsww.close()  

mrsos.piControl.E3SM-1-0.r1i1p1f1*
land mask wrong shape for mrsos.piControl.E3SM-1-0.r1i1p1f1


In [ ]:
landdata.getAxis(1).designateLatitude()
landdata.getAxis(2).designateLongitude()
landdata.shape

In [ ]:
Plotting.bmap(landdata[0])